# 문서 로더(document Loader)

In [2]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### Document 객체
: LangChain에서 Document 객체는 모든 데이터의 기본 단위이며, <br>
  Loader로 불러온 텍스트를 모델이 이해할 수 있는 표준 구조로 정리한 클래스

In [5]:
from langchain_core.documents import Document

doc = Document(
    page_content="이 문서는 LangChain의 Document 객체 설명입니다.",
    metadata={"source": "lecture_note", "page": 1}
)
print(type(doc))

print('page_content:',doc.page_content)
print('metadata:',doc.metadata)

<class 'langchain_core.documents.base.Document'>
page_content: 이 문서는 LangChain의 Document 객체 설명입니다.
metadata: {'source': 'lecture_note', 'page': 1}


In [6]:
docs = \
[
  Document(
    page_content="1장. LangChain 개요...",
    metadata={"source": "data/guide.pdf", "page": 1}
  ),
  Document(
    page_content="2장. Document Loader의 종류...",
    metadata={"source": "data/guide.pdf", "page": 2}
  ),
]
print(docs[0])
print(docs[1])

page_content='1장. LangChain 개요...' metadata={'source': 'data/guide.pdf', 'page': 1}
page_content='2장. Document Loader의 종류...' metadata={'source': 'data/guide.pdf', 'page': 2}


## 문서 로더(Document Loader)
: LangChain의 Document Loader는 외부 문서(파일, 웹, 데이터베이스 등)를 읽어서  <br>
LangChain이 처리할 수 있는 Document 객체(list[Document]) 형태로 변환하는 구성요소이다.

###  [1] TextLoader
: TextLoader 는 LangChain에서 가장 기본적인 문서 로더(Document Loader) 로, <br>
일반 텍스트 파일(.txt)을 읽어들여 Document 객체로 변환한다.

In [13]:
%%writefile sample.txt
이 문서는 LangChain TextLoader 예시입니다.
여러 줄의 텍스트를 포함합니다.
TextLoader 는 LangChain에서 가장 기본적인 Document Loader이다

Overwriting sample.txt


In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1) 텍스트 로드
loader = TextLoader("sample.txt",encoding="utf-8")
docs = loader.load()

print(type(doc))  # Document 객체
print(doc)

# 2) 청크 단위로 나누기
splitter = RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=10)
split_docs = splitter.split_documents(docs)

# 3) 임베딩 생성 및 벡터 저장
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(split_docs,embeddings)

print("총", len(split_docs), "개의 청크가 저장되었습니다.")
split_docs

<class 'langchain_core.documents.base.Document'>
page_content='이 문서는 LangChain의 Document 객체 설명입니다.' metadata={'source': 'lecture_note', 'page': 1}
총 4 개의 청크가 저장되었습니다.


[Document(metadata={'source': 'sample.txt'}, page_content='이 문서는 LangChain TextLoader 예시입니다.'),
 Document(metadata={'source': 'sample.txt'}, page_content='여러 줄의 텍스트를 포함합니다.'),
 Document(metadata={'source': 'sample.txt'}, page_content='TextLoader 는 LangChain에서 가장 기본적인 Document'),
 Document(metadata={'source': 'sample.txt'}, page_content='Document Loader이다')]

###  [2] DirectoryLoader
: DirectoryLoader는 폴더(디렉터리) 안의 여러 파일을 자동으로 탐색하여 <br>
각 파일을 개별 Document 객체로 읽어들이는 문서 로더(Document Loader) 이다. <br>
이 클래스는 실제 프로젝트에서 대규모 문서 일괄 처리 시 거의 항상 사용되는 핵심 도구이다.

In [17]:
! mkdir data

In [18]:
import os

# 1. data 폴더 생성 (이미 존재하면 무시)
os.makedirs("data", exist_ok=True)

# 2. 샘플 텍스트 데이터
samples = {
    "sample1.txt": """LangChain은 LLM(대규모 언어모델)을 활용한 파이프라인 구축을 위한 오픈소스 프레임워크입니다.
Document Loader를 통해 다양한 외부 데이터를 로드하고,
Text Splitter로 문서를 나눈 후, Embedding 및 VectorStore를 이용해 RAG 시스템을 구성할 수 있습니다.""",

    "sample2.txt": """FAISS는 Facebook AI Research에서 개발한 벡터 검색 라이브러리입니다.
LangChain에서는 문서 임베딩을 벡터로 변환한 후,
FAISS를 이용해 빠른 유사도 검색을 수행합니다.""",

    "sample3.txt": """pgvector는 PostgreSQL 데이터베이스에서 벡터 데이터를 저장하고 검색할 수 있게 해주는 확장 모듈입니다.
LangChain은 pgvector를 통해 RAG 시스템을 SQL 기반 환경에서도 구현할 수 있습니다."""
}

# 3. 파일 생성 및 저장
for filename, content in samples.items():
    path = os.path.join("data", filename)
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    print(f" {filename} 파일 생성 완료")

print("\n📂 'data' 폴더에 3개의 샘플 텍스트 파일이 준비되었습니다.")

 sample1.txt 파일 생성 완료
 sample2.txt 파일 생성 완료
 sample3.txt 파일 생성 완료

📂 'data' 폴더에 3개의 샘플 텍스트 파일이 준비되었습니다.


In [21]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader

loader = DirectoryLoader(
    "data/",
    glob="**/*.txt",
    loader_cls=lambda path : TextLoader(path,autodetect_encoding=True), #  자동 인코딩 감지
    show_progress=True
)

docs = loader.load()
print(len(docs), "개의 문서를 불러왔습니다.\n")

for i,doc in enumerate(docs):
    print(f'문서 {i+1}:\n', doc)
    print('-'*140)

 50%|███████████████████████████████████████████████████████████▌                                                           | 3/6 [00:00<00:00, 431.73it/s]

3 개의 문서를 불러왔습니다.

문서 1:
 page_content='LangChain은 LLM(대규모 언어모델)을 활용한 파이프라인 구축을 위한 오픈소스 프레임워크입니다.
Document Loader를 통해 다양한 외부 데이터를 로드하고,
Text Splitter로 문서를 나눈 후, Embedding 및 VectorStore를 이용해 RAG 시스템을 구성할 수 있습니다.' metadata={'source': 'data\\sample1.txt'}
--------------------------------------------------------------------------------------------------------------------------------------------
문서 2:
 page_content='FAISS는 Facebook AI Research에서 개발한 벡터 검색 라이브러리입니다.
LangChain에서는 문서 임베딩을 벡터로 변환한 후,
FAISS를 이용해 빠른 유사도 검색을 수행합니다.' metadata={'source': 'data\\sample2.txt'}
--------------------------------------------------------------------------------------------------------------------------------------------
문서 3:
 page_content='pgvector는 PostgreSQL 데이터베이스에서 벡터 데이터를 저장하고 검색할 수 있게 해주는 확장 모듈입니다.
LangChain은 pgvector를 통해 RAG 시스템을 SQL 기반 환경에서도 구현할 수 있습니다.' metadata={'source': 'data\\sample3.txt'}
---------------------------------------------------------------------------------------------------------

In [22]:
docs  # 리스트로 반환

[Document(metadata={'source': 'data\\sample1.txt'}, page_content='LangChain은 LLM(대규모 언어모델)을 활용한 파이프라인 구축을 위한 오픈소스 프레임워크입니다.\nDocument Loader를 통해 다양한 외부 데이터를 로드하고,\nText Splitter로 문서를 나눈 후, Embedding 및 VectorStore를 이용해 RAG 시스템을 구성할 수 있습니다.'),
 Document(metadata={'source': 'data\\sample2.txt'}, page_content='FAISS는 Facebook AI Research에서 개발한 벡터 검색 라이브러리입니다.\nLangChain에서는 문서 임베딩을 벡터로 변환한 후,\nFAISS를 이용해 빠른 유사도 검색을 수행합니다.'),
 Document(metadata={'source': 'data\\sample3.txt'}, page_content='pgvector는 PostgreSQL 데이터베이스에서 벡터 데이터를 저장하고 검색할 수 있게 해주는 확장 모듈입니다.\nLangChain은 pgvector를 통해 RAG 시스템을 SQL 기반 환경에서도 구현할 수 있습니다.')]

### [3] PDF Loader

####  (1) PyPDFLoader
: PyPDFLoader는 PDF 파일을 텍스트로 변환하여 Document 객체로 만드는 로더입니다.<br>
내부적으로 pypdf (이전 명칭 PyPDF2) 라이브러리를 사용하며,
PDF의 각 페이지를 개별 Document 객체로 반환합니다.

In [23]:
# ! pip install pypdf

In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")
docs = loader.load()

print("총", len(docs), "페이지 로드됨")
print(docs[0].page_content[:500])

총 16 페이지 로드됨
강화학습을 이용한 하천 녹조 발생 저감 모형 연구   47한국빅데이터학회지
제10권 제1호, 2025, pp. 47-62 https://doi.org/10.36498/kbigdt.2025.10.1.47
유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지: 
사용자 생성 콘텐츠에서의 극단성과 허위성 분류 
Decoding Review Anomalies: Classifying Extremity and Falsity in 
User-Generated Content
가중정1⋅김엘레나2⋅최재원3†
순천향대학교 경영학과1, 순천향대학교 경영학과2, 순천향대학교 경영학과3
요  약
본 연구는 YouTube 플랫폼의 호텔 리뷰를 대상으로 머신러닝과 자연어처리(NLP) 기법을 활용해 극단적 
및 조작된 리뷰를 식별⋅필터링하고자 한다. 소셜 미디어는 소비자 구매 결정에 중요한 영향을 미치며, 
사용자 생성 리뷰는 마켓플레이스 신뢰도의 핵심 요소로 작용한다. 그러나 일부 판매자들의 평점 조작과 
조작된 리


In [25]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader(
    "./",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)
docs = loader.load()

print("총", len(docs), "개의 문서(페이지) 로드 완료")

총 78 개의 문서(페이지) 로드 완료


In [27]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1) PDF 로드
loader = PyPDFLoader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")
docs = loader.load()

# 2) 문서 분할
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

# 3) 임베딩 및 벡터 저장
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(split_docs, embeddings)

print("PDF 문서가 벡터 DB로 성공적으로 저장되었습니다.")

PDF 문서가 벡터 DB로 성공적으로 저장되었습니다.


In [28]:
len(split_docs)

58

 #### (2) PyMuPDF

In [29]:
# !pip install pymupdf

In [31]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")

# 문서 로드
docs = loader.load()

print("총", len(docs), "페이지 로드됨")

# 문서의 내용 출력
print(docs[10].page_content[:500])

총 16 페이지 로드됨
강화학습을 이용한 하천 녹조 발생 저감 모형 연구   47
한국빅데이터학회지
제10권 제1호, 2025, pp. 47-62
https://doi.org/10.36498/kbigdt.2025.10.1.47
유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지: 
사용자 생성 콘텐츠에서의 극단성과 허위성 분류 
Decoding Review Anomalies: Classifying Extremity and Falsity in 
User-Generated Content
가중정1⋅김엘레나2⋅최재원3†
순천향대학교 경영학과1, 순천향대학교 경영학과2, 순천향대학교 경영학과3
요  약
본 연구는 YouTube 플랫폼의 호텔 리뷰를 대상으로 머신러닝과 자연어처리(NLP) 기법을 활용해 극단적 
및 조작된 리뷰를 식별⋅필터링하고자 한다. 소셜 미디어는 소비자 구매 결정에 중요한 영향을 미치며, 
사용자 생성 리뷰는 마켓플레이스 신뢰도의 핵심 요소로 작용한다. 그러나 일부 판매자들의 평점 조작과 
조작된 


#### (4) PDFPlumber

In [32]:
# !pip install pdfplumber

In [34]:
from langchain_community.document_loaders import PDFPlumberLoader

# PDF 문서 로더 인스턴스 생성
loader = PDFPlumberLoader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")

# 문서 로딩
docs = loader.load()
print("총", len(docs), "페이지 로드됨")

# 첫 번째 문서 데이터 접근
print(docs[10].page_content[:500])

총 16 페이지 로드됨
유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지 57
추출된 토픽 주제의 해석력을 높이기 위해, 동시출현 네트워크는 사용자 리뷰의 다차원
본 연구는 동시출현 점수(co-occurrence score)를 적인 토픽 구조를 보여준다. 다양한 색상의 토
계산하여 각 토픽의 의미적 일관성을 평가하였 픽 노드는 여행 및 숙박 경험, 지역 리뷰, 동영상
다. 이 점수는 특정 주제 내에서 키워드들이 동 콘텐츠 피드백 등과 같은 다양한 의미적 시나리
일한 문서 내에 얼마나 자주 함께 등장하는지를 오를 다루며, 고빈도 키워드는 여러 토픽을 동
수치화한 것으로, 주제어 간의 상호 연관성을 시 발생 관계를 통해 서로 연결하여 리뷰 콘텐
측정하는 지표로 활용된다. 수치화한 것으로, 주 츠의 다양성과 집중도를 보여준다. <그림 2>는
제어 간의 상호 연관성을 측정하는 지표로 활용 시각화 된 동시출현 네트워크를 나타낸다.
된다. 동시출현 점수가 높을수록 키워드들이 밀 Word Cloud 시각화는 사용 시나리


#### (5) PyPDFium2

In [35]:
from langchain_community.document_loaders import PyPDFium2Loader

# PyPDFium2 로더 인스턴스 생성
loader = PyPDFium2Loader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")

# 데이터 로드
docs = loader.load()
print("총", len(docs), "페이지 로드됨")

# 문서의 내용 출력
print(docs[10].page_content[:500])

총 16 페이지 로드됨
유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지 57
추출된 토픽 주제의 해석력을 높이기 위해, 
본 연구는 동시출현 점수(co-occurrence score)를 
계산하여 각 토픽의 의미적 일관성을 평가하였
다. 이 점수는 특정 주제 내에서 키워드들이 동
일한 문서 내에 얼마나 자주 함께 등장하는지를 
수치화한 것으로, 주제어 간의 상호 연관성을 
측정하는 지표로 활용된다. 수치화한 것으로, 주
제어 간의 상호 연관성을 측정하는 지표로 활용
된다. 동시출현 점수가 높을수록 키워드들이 밀
접하게 연결되어 있는 일관된 주제를 형성하며, 
예를 들어 ‘호텔 시설’, ‘청결’, ‘침대’, ‘조식’ 등
이 함께 자주 등장하는 경우, 해당 리뷰는 특정 
호텔의 물리적 조건에 집중된 논의임을 시사한
다. 반면, 동시출현 점수가 낮은 토픽은 단어 간 
맥락적 연결성이 약하거나, 리뷰 작성자들이 다
양한 맥락에서 단어를 더 자유롭게 사용하는 경
향이 있음을 나타낸다. 이는 주제가 분산되어 
있거나


### [4] CSVLoader 와 DataFrameLoader

In [40]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성
loader = CSVLoader(file_path= "WHO_first9cols.csv")
docs = loader.load()

print(len(docs))
print(docs[0].metadata)
print(docs[0])

202
{'source': 'WHO_first9cols.csv', 'row': 0}
page_content='Country: Afghanistan
CountryID: 1
Continent: 1
Adolescent fertility rate (%): 151
Adult literacy rate (%): 28
Gross national income per capita (PPP international $): 
Net primary school enrolment ratio female (%): 
Net primary school enrolment ratio male (%): 
Population (in thousands) total: 26088' metadata={'source': 'WHO_first9cols.csv', 'row': 0}


In [38]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("WHO_first9cols.csv")
df

,Country,CountryID,Continent,Adolescent fertility rate (%),Adult literacy rate (%),Gross national income per capita (PPP international $),Net primary school enrolment ratio female (%),Net primary school enrolment ratio male (%),Population (in thousands) total
0,Afghanistan,1,1,151.0,28.0,NaN,NaN,NaN,26088.0
1,Albania,2,2,27.0,98.7,6000.0,93.0,94.0,3172.0
2,Algeria,3,3,6.0,69.9,5940.0,94.0,96.0,33351.0
3,Andorra,4,2,NaN,NaN,NaN,83.0,83.0,74.0
4,Angola,5,3,146.0,67.4,3890.0,49.0,51.0,16557.0
...,...,...,...,...,...,...,...,...,...
197,Vietnam,198,6,25.0,90.3,2310.0,91.0,96.0,86206.0
198,West Bank and Gaza,199,1,NaN,NaN,NaN,NaN,NaN,NaN
199,Yemen,200,1,83.0,54.1,2090.0,65.0,85.0,21732.0
200,Zambia,201,3,161.0,68.0,1140.0,94.0,90.0,11696.0


In [41]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(df, page_content_column="Country")

# 문서 로드
docs = loader.load()

# 데이터 출력
print(docs[0].page_content)

# 메타데이터 출력
print(docs[0].metadata)

Afghanistan
{'CountryID': 1, 'Continent': 1, 'Adolescent fertility rate (%)': 151.0, 'Adult literacy rate (%)': 28.0, 'Gross national income per capita (PPP international $)': nan, 'Net primary school enrolment ratio female (%)': nan, 'Net primary school enrolment ratio male (%)': nan, 'Population (in thousands) total': 26088.0}


### [5] JSONLoader

In [42]:
%%writefile employee.json
[
    {
        "name": "홍길동",
        "age": 30,
        "address": "서울특별시 강남구",
        "phoneNumbers": [
            {
                "type": "mobile",
                "number": "010-1234-5678"
            },
            {
                "type": "home",
                "number": "02-987-6543"
            }
        ]
    },
    {
        "name": "김철수",
        "age": 25,
        "address": "부산광역시 해운대구",
        "phoneNumbers": [
            {
                "type": "mobile",
                "number": "010-1111-2222"
            }
        ]
    },
    {
        "name": "이영희",
        "age": 28,
        "address": "대구광역시 수성구",
        "phoneNumbers": [
            {
                "type": "mobile",
                "number": "010-3333-4444"
            },
            {
                "type": "work",
                "number": "053-555-6666"
            }
        ]
    }
]

Writing employee.json


In [43]:
# !pip install jq

In [44]:
from langchain_community.document_loaders import JSONLoader

# JSONLoader 생성
loader = JSONLoader(
    file_path="employee.json",
    jq_schema=".[].phoneNumbers",
    text_content=False,
)

# 문서 로드
docs = loader.load()

# 결과 출력
docs


[Document(metadata={'source': 'C:\\Users\\storm\\바탕 화면\\AI_Agent_Lab_PM\\04_LangChain 기본\\employee.json', 'seq_num': 1}, page_content='[{"type": "mobile", "number": "010-1234-5678"}, {"type": "home", "number": "02-987-6543"}]'),
 Document(metadata={'source': 'C:\\Users\\storm\\바탕 화면\\AI_Agent_Lab_PM\\04_LangChain 기본\\employee.json', 'seq_num': 2}, page_content='[{"type": "mobile", "number": "010-1111-2222"}]'),
 Document(metadata={'source': 'C:\\Users\\storm\\바탕 화면\\AI_Agent_Lab_PM\\04_LangChain 기본\\employee.json', 'seq_num': 3}, page_content='[{"type": "mobile", "number": "010-3333-4444"}, {"type": "work", "number": "053-555-6666"}]')]